In [1]:
import sys 
import cv2
import os
from os.path import isfile, join
import pandas as pd
from youtubesearchpython import SearchVideos
import pandas as pd
from lxml import html, etree
import pafy
from youtube_transcript_api import YouTubeTranscriptApi
import json
import os

In [2]:
# find a good video:

#https://medium.com/@iKhushPatel/convert-video-to-images-images-to-video-using-opencv-python-db27a128a481
#vidcap.get(cv2.CAP_PROP_FPS)
#vidcap.get(cv2.CAP_PROP_POS_MSEC)

# for some reason, openCV failed to open the video from url, need to download and read!

url = 'https://www.youtube.com/watch?v=SVENlyTNdpU'
video = pafy.new(url)
bestResolutionVideo = video.getbest()
bestResolutionVideo.download()

  70,584,938.0 Bytes [100.00%] received. Rate: [19759 KB/s].  ETA: [0 secs]    


In [4]:
vidcap = cv2.VideoCapture('g.mp4')
#vidcap.get(cv2.CAP_PROP_FPS)
vidcap.get(cv2.CAP_PROP_POS_MSEC)

0.0

In [3]:
# video - to -images 
vidcap = cv2.VideoCapture('a.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)   #set the capturing start at (sec*1000 milliseconds)
    hasFrames,image = vidcap.read()
    if hasFrames:
        os.chdir(r'C:\Users\DrHeh\Google Drive\programming\py_InsightFellowProject\pre-course prep\video-to-image8')
        cv2.imwrite("image"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames


if __name__ == '__main__':
    sec = 0 #capture start at 120 seconds 
    #frameRate = 1/vidcap.get(cv2.CAP_PROP_FPS) #//it will capture image in each "frameRate" second
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1  #for label images 
        sec = sec + 1 #every 1 sec 
        #sec = sec + frameRate*100 #every 100 frames
        sec = round(sec, 2)
        success = getFrame(sec)

In [55]:
# images - to -videos 
pathIn= 'C:/Users/DrHeh/OneDrive/Documents/Insight Data Science Program/Jupyter python scripts/video1Frames 5 models run/outputAllFrame_replacedMaskFrame/'
pathOut = r'C:\Users\DrHeh\Google Drive\programming\py_InsightFellowProject\FobiaPhilter(scripts drafted local)\Buffered_converted_videos\video1.mp4'  #or .mp4
df_filename_imageID = pd.read_csv(r'C:\Users\DrHeh\OneDrive\Documents\Insight Data Science Program\Jupyter python scripts\video1Frames 5 models run\video1filenames.txt')
fps = 0.5
   

# use 00100 model and align with the origin image name with the right order 
selected_files = []
for file in os.listdir(pathIn):
    if file.endswith('_0050.h5.jpg'):
        selected_files.append(file) 

# fix the order
df = pd.DataFrame({'outputFile': selected_files})
df['Index_outputFile'] = df['outputFile'].str.replace('_mask_rcnn_clown_cfg_0050.h5.jpg','')
df['Index_outputFile'] = df['Index_outputFile'].astype('int64')
dfIndex = df_filename_imageID.rename(columns={'Unnamed: 0':"Index_outputFile", '0':"RightOrderOriginalFile"})
dfIndex['RightOrderOriginalFile'] = dfIndex['RightOrderOriginalFile'].str.replace('dataset.image','')
dfIndex['RightOrderOriginalFile'] = dfIndex['RightOrderOriginalFile'].astype('int64')
Use_File = pd.merge(df, dfIndex, on=['Index_outputFile'], how='outer').sort_values(by=['RightOrderOriginalFile'])
Use_File = Use_File.iloc[586:623]

# start to convert images to videos
frame_array = []
for file in Use_File['outputFile']:
    filename=pathIn + file
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [10]:
# To play video 
path = r'C:\Users\DrHeh\OneDrive\Documents\Insight Data Science Program\Jupyter python scripts\FobiaPhilter//'
cap = cv2.VideoCapture(path + 'video1.mp4')

while True:
    ret, frame = cap.read()
    if ret == True:
        frame2 = cv2.resize(frame, (1600, 800))
        cv2.imshow('frame', frame2)
        cv2.waitKey(10000)
        
        if cv2.waitKey(10000) & 0xFF == ord('q'):   # Press Q on keyboard to  exit 
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()